## Benchmarking

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Benchmarking import *
import sys

/home/arvindsk/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
chm = 20

In [4]:
user = "arvindsk"
# user = "wknd37"

datasets_benchmark = "/home/" + user + "/xgmix_expts/benchmark_data/val/chm{0}/{1}/{2}/generated_data/"
metrics_benchmark = "/home/" + user + "/xgmix_expts/benchmark_data/val/chm{0}/{1}/{2}/generated_data/chm{0}/"
model_root = "/home/" + user + "/xgmix_expts/benchmark_data/{1}/{2}/{3}/generated_data/chm{0}/all/models/{4}_{5}.pkl"
train_data_root = "/home/" + user + "/xgmix_expts/benchmark_data/{0}/{1}/{2}/generated_data/"

train_gens = [0,2,4,8,16,24,32,48,64,72,100]
val_gens = [2,4,8,12,16,20,30,40,50,60,70,80,90,100]

In [5]:
verbose=True

In [6]:
for set_name in ["full","ukb"]:
    W = 30 if set_name == "ukb" else 1000

    for pop_name in ["latino","five","seven"]:
        
        data_path = datasets_benchmark.format(chm,set_name,pop_name)
        print("Val data from: ",data_path)
        

        for base in ["lgb", "xgb", "logreg"]:
            for smooth in ["xgb", "crf", "cnv"]:
                
                for bal in ["unbal_unbal","bal_admix"]:

                    metric_path = os.path.join(metrics_benchmark.format(chm,set_name,pop_name),"{}_{}_{}.benchmark".format(base,smooth,bal))
                    print("Metrics will show up at: ",metric_path)
                    metrics = {}

                    # load model
                    model_path = model_root.format(chm,set_name,bal,pop_name,base,smooth)
                    print("Loading model from: ",model_path)
                    
                    # load the original train and val data to get training time metrics
                    print("Getting traintime data")
                    train_data_path = train_data_root.format(set_name,bal,pop_name)
                    data, meta, _ = get_data(train_data_path, W=W, gens=train_gens, chm=chm, verbose=False)
                    (X_t1, y_t1), (X_t2, y_t2), (X_v, y_v) = data
                    
                    print("Getting validation set that was separately generated")
                    print("Val generations: ",val_gens)
                    val_paths = [data_path + "/chm{}/simulation_output/val/gen_".format(chm)    + str(gen) + "/" for gen in val_gens] 
                    X_val_files    = [p + "mat_vcf_2d.npy" for p in val_paths]
                    labels_val_files    = [p + "mat_map.npy" for p in val_paths]
                    train_val_files = [X_val_files, labels_val_files]
                    X_val_raw, labels_val_raw = [load_np_data(f) for f in train_val_files]
                    X_val, labels_window_val       = data_process(X_val_raw, labels_val_raw, W, 0)
                    val = np.array(X_val).astype("int8")
                    val_lab = np.array(labels_window_val).astype("int16")
                    
                    # glue the old train data with the new val data
                    data = ((X_t1,y_t1),(X_t2,y_t2),(val,val_lab))
                    y_snp = labels_val_raw.copy()
                    
                    # here pass on the val_gens since we only pass val set into the accuracy computing functions
                    metrics = bm_eval(model_path, data, gens=val_gens, eval_calibration=True, y_snp=y_snp, verbose=verbose)

                    save_dict(metrics,metric_path)

Val data from:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/full/latino/generated_data/
Metrics will show up at:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/full/latino/generated_data/chm20/lgb_xgb_unbal_unbal.benchmark
Loading model from:  /home/arvindsk/xgmix_expts/benchmark_data/full/unbal_unbal/latino/generated_data/chm20/all/models/lgb_xgb.pkl
Getting traintime data
Getting validation set that was separately generated
Val generations:  [2, 4, 8, 12, 16, 20, 30, 40, 50, 60, 70, 80, 90, 100]
Base Training Accuracy:   97.67%
Base Validation Accuracy: 95.54%
Smooth Training Accuracy: 98.47%
Smooth Validation Accuracy: 97.55%
retrieving accuracies..
calculating log loss..
retrieving training and inference time..
estimating model size..
estimating accuracy for each generation..
Smooth Training Accuracy: 98.46%
Smooth Validation Accuracy: 97.53%
Metrics will show up at:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/full/latino/generated_data/chm20/lgb_xgb_bal_admi

Base Training Accuracy:   97.69%
Base Validation Accuracy: 95.38%
Smooth Training Accuracy: 97.6%
Smooth Validation Accuracy: 96.7%
retrieving accuracies..
calculating log loss..
retrieving training and inference time..
estimating model size..
estimating accuracy for each generation..
Smooth Training Accuracy: 97.64%
Smooth Validation Accuracy: 96.75%
Metrics will show up at:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/full/latino/generated_data/chm20/xgb_cnv_bal_admix.benchmark
Loading model from:  /home/arvindsk/xgmix_expts/benchmark_data/full/bal_admix/latino/generated_data/chm20/all/models/xgb_cnv.pkl
Getting traintime data
Getting validation set that was separately generated
Val generations:  [2, 4, 8, 12, 16, 20, 30, 40, 50, 60, 70, 80, 90, 100]
Base Training Accuracy:   96.85%
Base Validation Accuracy: 95.08%
Smooth Training Accuracy: 97.01%
Smooth Validation Accuracy: 96.67%
retrieving accuracies..
calculating log loss..
retrieving training and inference time..
estimat

Metrics will show up at:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/full/five/generated_data/chm20/lgb_crf_bal_admix.benchmark
Loading model from:  /home/arvindsk/xgmix_expts/benchmark_data/full/bal_admix/five/generated_data/chm20/all/models/lgb_crf.pkl
Getting traintime data
Getting validation set that was separately generated
Val generations:  [2, 4, 8, 12, 16, 20, 30, 40, 50, 60, 70, 80, 90, 100]
Base Training Accuracy:   88.5%
Base Validation Accuracy: 85.61%
Smooth Training Accuracy: 94.61%
Smooth Validation Accuracy: 93.46%
retrieving accuracies..
calculating log loss..
retrieving training and inference time..
estimating model size..
estimating accuracy for each generation..
Smooth Training Accuracy: 94.61%
Smooth Validation Accuracy: 93.36%
Metrics will show up at:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/full/five/generated_data/chm20/lgb_cnv_unbal_unbal.benchmark
Loading model from:  /home/arvindsk/xgmix_expts/benchmark_data/full/unbal_unbal/five/generate

retrieving training and inference time..
estimating model size..
estimating accuracy for each generation..
Smooth Training Accuracy: 98.11%
Smooth Validation Accuracy: 96.04%
Metrics will show up at:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/full/five/generated_data/chm20/logreg_crf_unbal_unbal.benchmark
Loading model from:  /home/arvindsk/xgmix_expts/benchmark_data/full/unbal_unbal/five/generated_data/chm20/all/models/logreg_crf.pkl
Getting traintime data
Getting validation set that was separately generated
Val generations:  [2, 4, 8, 12, 16, 20, 30, 40, 50, 60, 70, 80, 90, 100]
Base Training Accuracy:   97.97%
Base Validation Accuracy: 91.62%
Smooth Training Accuracy: 98.84%
Smooth Validation Accuracy: 95.33%
retrieving accuracies..
calculating log loss..
retrieving training and inference time..
estimating model size..
estimating accuracy for each generation..
Smooth Training Accuracy: 98.83%
Smooth Validation Accuracy: 95.24%
Metrics will show up at:  /home/arvindsk/xgmix

Getting validation set that was separately generated
Val generations:  [2, 4, 8, 12, 16, 20, 30, 40, 50, 60, 70, 80, 90, 100]
Base Training Accuracy:   86.28%
Base Validation Accuracy: 80.16%
Smooth Training Accuracy: 94.91%
Smooth Validation Accuracy: 91.43%
retrieving accuracies..
calculating log loss..
retrieving training and inference time..
estimating model size..
estimating accuracy for each generation..
Smooth Training Accuracy: 94.92%
Smooth Validation Accuracy: 91.4%
Metrics will show up at:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/full/seven/generated_data/chm20/xgb_xgb_bal_admix.benchmark
Loading model from:  /home/arvindsk/xgmix_expts/benchmark_data/full/bal_admix/seven/generated_data/chm20/all/models/xgb_xgb.pkl
Getting traintime data
Getting validation set that was separately generated
Val generations:  [2, 4, 8, 12, 16, 20, 30, 40, 50, 60, 70, 80, 90, 100]
Base Training Accuracy:   84.77%
Base Validation Accuracy: 79.94%
Smooth Training Accuracy: 92.82%
Smoot

...

Base Training Accuracy:   90.5%
Base Validation Accuracy: 89.32%
Smooth Training Accuracy: 95.9%
Smooth Validation Accuracy: 95.81%
retrieving accuracies..
calculating log loss..
retrieving training and inference time..
estimating model size..
estimating accuracy for each generation..
Smooth Training Accuracy: 96.02%
Smooth Validation Accuracy: 95.85%
Metrics will show up at:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/ukb/latino/generated_data/chm20/xgb_cnv_unbal_unbal.benchmark
Loading model from:  /home/arvindsk/xgmix_expts/benchmark_data/ukb/unbal_unbal/latino/generated_data/chm20/all/models/xgb_cnv.pkl
Getting traintime data
Getting validation set that was separately generated
Val generations:  [2, 4, 8, 12, 16, 20, 30, 40, 50, 60, 70, 80, 90, 100]
Base Training Accuracy:   92.45%
Base Validation Accuracy: 89.81%
Smooth Training Accuracy: 96.21%
Smooth Validation Accuracy: 95.19%
retrieving accuracies..
calculating log loss..
retrieving training and inference time..
estim

Metrics will show up at:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/ukb/five/generated_data/chm20/lgb_crf_unbal_unbal.benchmark
Loading model from:  /home/arvindsk/xgmix_expts/benchmark_data/ukb/unbal_unbal/five/generated_data/chm20/all/models/lgb_crf.pkl
Getting traintime data
Getting validation set that was separately generated
Val generations:  [2, 4, 8, 12, 16, 20, 30, 40, 50, 60, 70, 80, 90, 100]
Base Training Accuracy:   79.3%
Base Validation Accuracy: 75.41%
Smooth Training Accuracy: 92.69%
Smooth Validation Accuracy: 89.9%
retrieving accuracies..
calculating log loss..
retrieving training and inference time..
estimating model size..
estimating accuracy for each generation..
Smooth Training Accuracy: 92.94%
Smooth Validation Accuracy: 90.08%
Metrics will show up at:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/ukb/five/generated_data/chm20/lgb_crf_bal_admix.benchmark
Loading model from:  /home/arvindsk/xgmix_expts/benchmark_data/ukb/bal_admix/five/generated_dat

retrieving training and inference time..
estimating model size..
estimating accuracy for each generation..
Smooth Training Accuracy: 93.07%
Smooth Validation Accuracy: 90.71%
Metrics will show up at:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/ukb/five/generated_data/chm20/logreg_xgb_bal_admix.benchmark
Loading model from:  /home/arvindsk/xgmix_expts/benchmark_data/ukb/bal_admix/five/generated_data/chm20/all/models/logreg_xgb.pkl
Getting traintime data
Getting validation set that was separately generated
Val generations:  [2, 4, 8, 12, 16, 20, 30, 40, 50, 60, 70, 80, 90, 100]
Base Training Accuracy:   74.76%
Base Validation Accuracy: 74.49%
Smooth Training Accuracy: 90.96%
Smooth Validation Accuracy: 90.2%
retrieving accuracies..
calculating log loss..
retrieving training and inference time..
estimating model size..
estimating accuracy for each generation..
Smooth Training Accuracy: 91.03%
Smooth Validation Accuracy: 90.52%
Metrics will show up at:  /home/arvindsk/xgmix_expts/

Getting validation set that was separately generated
Val generations:  [2, 4, 8, 12, 16, 20, 30, 40, 50, 60, 70, 80, 90, 100]
Base Training Accuracy:   70.48%
Base Validation Accuracy: 68.08%
Smooth Training Accuracy: 89.06%
Smooth Validation Accuracy: 86.76%
retrieving accuracies..
calculating log loss..
retrieving training and inference time..
estimating model size..
estimating accuracy for each generation..
Smooth Training Accuracy: 89.05%
Smooth Validation Accuracy: 86.75%
Metrics will show up at:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/ukb/seven/generated_data/chm20/xgb_xgb_unbal_unbal.benchmark
Loading model from:  /home/arvindsk/xgmix_expts/benchmark_data/ukb/unbal_unbal/seven/generated_data/chm20/all/models/xgb_xgb.pkl
Getting traintime data
Getting validation set that was separately generated
Val generations:  [2, 4, 8, 12, 16, 20, 30, 40, 50, 60, 70, 80, 90, 100]
Base Training Accuracy:   74.81%
Base Validation Accuracy: 70.53%
Smooth Training Accuracy: 91.21%
Sm

Metrics will show up at:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/ukb/seven/generated_data/chm20/logreg_cnv_unbal_unbal.benchmark
Loading model from:  /home/arvindsk/xgmix_expts/benchmark_data/ukb/unbal_unbal/seven/generated_data/chm20/all/models/logreg_cnv.pkl
Getting traintime data
Getting validation set that was separately generated
Val generations:  [2, 4, 8, 12, 16, 20, 30, 40, 50, 60, 70, 80, 90, 100]
Base Training Accuracy:   76.22%
Base Validation Accuracy: 72.86%
Smooth Training Accuracy: 90.96%
Smooth Validation Accuracy: 87.38%
retrieving accuracies..
calculating log loss..
retrieving training and inference time..
estimating model size..
estimating accuracy for each generation..
Smooth Training Accuracy: 91.0%
Smooth Validation Accuracy: 87.43%
Metrics will show up at:  /home/arvindsk/xgmix_expts/benchmark_data/val/chm20/ukb/seven/generated_data/chm20/logreg_cnv_bal_admix.benchmark
Loading model from:  /home/arvindsk/xgmix_expts/benchmark_data/ukb/bal_admix/seven

In [7]:
metrics

{'base_train_acc': 68.58,
 'base_val_acc': 67.9,
 'base_train_acc_bal': 67.64,
 'base_val_acc_bal': 61.1,
 'train_acc': 88.14,
 'val_acc': 87.09,
 'train_acc_bal': 88.14,
 'val_acc_bal': 81.19,
 'log_loss': 0.39,
 'smooth_training_time': 82.8,
 'smooth_inference_time': 0.05,
 'training_time': 1190.6699999999998,
 'inference_time': 0.29,
 'model_total_size_mb': 2.06,
 'gen_performance': {'gens': [2,
   4,
   8,
   12,
   16,
   20,
   30,
   40,
   50,
   60,
   70,
   80,
   90,
   100],
  'accs': [96.65,
   96.08,
   95.14,
   91.96,
   93.27,
   90.2,
   89.11,
   86.68,
   84.66,
   82.05,
   80.31,
   79.45,
   77.75,
   75.9],
  'accs_snp_lvl': [96.64,
   96.06,
   95.12,
   91.93,
   93.21,
   90.08,
   88.95,
   86.52,
   84.37,
   81.78,
   80.03,
   79.09,
   77.39,
   75.54]},
 'val_acc_snp_lvl': 86.91,
 'val_acc_cal': 87.12,
 'val_ll_cal': 0.38641329680434583}

In [ ]:
def run_bm(base, smooth, chm, gen_expts, models_exist=False, local_metric=None):

    for data_path in datasets_benchmark:

        # data_path must end with generated_data
        data_path = data_path + "generated_data/"

        for gen_name in gen_expts:

            bm_root = data_path + "chm{}/{}/".format(chm,gen_name)
            snp, bal, dat = np.array(bm_root.split("/"))[-7:-4]
            
            if snp == "full":
                W = 1000
            elif snp== "ukb":
                W = 30
            
            metric_path = bm_root + "benchmark_results.pkl"
            if local_metric is not None:
                metric_path = metric_path.replace("arvindsk", local_metric)

            gens = gen_expts[gen_name]

            print("-"*100)
            print("Experiment details")
            print("Dataset used: {}".format(data_path))
            print("Generations used: {}".format(gens))
            print("Metrics will come up at: {}".format(metric_path))

            metrics = bm_train(base, smooth, 
                               bm_root, data_path, 
                               gens, chm=chm,
                               W=W, 
                               load_base=True, load_smooth=True, 
                               eval=False,
                               models_exist=models_exist,
                               verbose=True)

            save_dict(metrics, metric_path)
            

In [ ]:
base = ["lgb", "xgb", "logreg"]
smooth = ["xgb", "crf", "cnv"]

# here we run 3 base, 3 smooth, 2 datasets, 3 populations, 2 balance

chm = 20
gen_expts = {
    "all" : [0,2,4,8,16,24,32,48,64,72,100]
}
run_bm(base, smooth, chm, gen_expts, models_exist=False, local_metric="arvindsk")

In [ ]:
base = ["rf","lgb", "xgb", "logreg"]
smooth = ["xgb", "crf", "cnv"]
for gen_name in gen_expts:
    for data_path in datasets_benchmark:
        bm_root = data_path + "generated_data/chm{}/{}/".format(chm,gen_name)
        print(bm_root)
        metric_path = bm_root + "benchmark_results.pkl"
        M = load_dict(metric_path)
        accs = plt_metric(base ,smooth, M, "val_acc")

In [ ]:
import seaborn as sns; 
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

def get_metric(base,smooth,Metrics,metric):
    M = {}
    for i, b in enumerate(base):
        M[b] = {}
        for j, s in enumerate(smooth):
            M[b][s] = Metrics[b+"_"+s][metric]
            
    return pd.DataFrame(M)


def plt_metric(base,smooth,Metrics,metric,vminmax=(None,None),cbar=True):
    
    if metric == "gen_performance":
        cmap = { 0:'k', 1:'b',  2:'y', 3:'g'}
        lmap = { 0:'-', 1:'--', 2:':', 3:'-.' }
        v_accs = get_metric(base, smooth, Metrics, "val_acc")
        for i, b in enumerate(base):
            for j, s in enumerate(smooth):
                m_name = b + "_" + s
                data = Metrics[m_name][metric] 
                plt.plot(data["gens"], data["accs"], linestyle=lmap[i%len(lmap)], color=cmap[j%len(cmap)],
                         label= m_name + " (" + str(v_accs[b][j]) + "%)")

        plt.legend(bbox_to_anchor=(1, 1), loc='upper left') 
        plt.xlabel("Generation")
        plt.ylabel("Accuracy (%)")
        plt.show()

    else:
        data = get_metric(base,smooth,Metrics,metric)
        sns.heatmap(data, annot=True, fmt=".2f",vmin=vminmax[0], vmax=vminmax[1], cbar=cbar)
        plt.xlabel("base model")
        plt.ylabel("smoother")
        plt.show()
        
    return data

In [ ]:
base = ["rf", "lgb", "xgb", "logreg"]
smooth = ["cnv", "xgb", "crf"]

cols = ["base", "smooth","data","gen","bal","snp","val_acc"]
ACC = [] # (b, s, data, gen, bal, ukb, acc)
all_accs = np.zeros( (len(base), len(smooth), len(gen_expts), len(datasets_benchmark)) )
for g, gen_name in enumerate(gen_expts):
    for d, data_path in enumerate(datasets_benchmark):
        bm_root = data_path + "generated_data/chm{}/{}/".format(chm,gen_name)
        metric_path = bm_root + "benchmark_results.pkl"
        M = load_dict(metric_path)
        
        v_accs = get_metric(base, smooth, M, "val_acc")
        
        dat = np.array(bm_root.split("/"))[-5]
        bal = np.array(bm_root.split("/"))[-6]
        ukb = np.array(bm_root.split("/"))[-7]
        for b in base:
            for s in smooth:
                ACC += [(b,s,dat,gen_name,bal,ukb,v_accs[b][s])]
DF = pd.DataFrame(ACC, columns=cols)
DF

## (LogReg) X (XGB, CRF) performing best

In [ ]:
DF.boxplot(column=["val_acc"], by=["snp","base", "smooth"], figsize=(16,8), rot="45")
plt.show()

In [ ]:
DF.boxplot(column=["val_acc"], by=["snp", "base", "smooth"], figsize=(16,8), rot="45")
plt.show()

## Closer look

In [ ]:
DF_best_models = DF[ 
    ~(DF.base.isin(["rf", "lgb"])) &
    (DF.smooth != "cnv")
]
DF_best_models.boxplot(column=["val_acc"], by=["base", "smooth"], figsize=(16,8), rot="45")
plt.show()

## Balance not affacting much but still a bit
we want to repeat with fixed validation set

In [ ]:
DF.boxplot(column=["val_acc"], by=["data", "bal"], figsize=(16,8), rot=45)
plt.show()

## same pattern using only best models

In [ ]:
DF_best_models.boxplot(column=["val_acc"], by=["data", "bal"], figsize=(16,8), rot=45)
plt.show()

## Accuracies similar for high generations

In [ ]:
DF_lathi = DF[ 
    (DF.base != "rf") &
    (DF.smooth != "cnv") &
    (DF.gen=="high") ]
DF_lathi.boxplot(column=["val_acc"], by=["data", "snp", "smooth"], figsize=(16,8), rot=45)
plt.show()

In [ ]:
data_path = "/home/arvindsk/xgmix_expts/benchmark_data/ukb/unbal_unbal/five/"
base = ["xgb", "logreg"]
smooth = ["xgb", "crf"]
gen_name = "high"
bm_root = data_path + "generated_data/chm{}/{}/".format(chm,gen_name)
metric_path = bm_root + "benchmark_results.pkl"
M = load_dict(metric_path)
out = plt_metric(base ,smooth, M, "gen_performance")

## Note! acc > 90% for gen 100